### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from neuropy import plotting
from scipy.ndimage import gaussian_filter1d
from neuropy import plotting
from scipy import stats
import subjects
import pandas as pd

### Firing rate stability and saving in separate file
- something like Miyawaki et al. 2016

In [2]:
# sessions = subjects.mua_sess()
sessions = subjects.sdrol.allsess
frate_all=[]
for sub, sess in enumerate(sessions):
    pre = sess.paradigm["pre"].flatten()
    post = sess.paradigm["post"].flatten()
    # neurons = sess.neurons.get_neuron_type(["pyr",'inter','mua'])
    neurons = sess.neurons
    
    rec_duration = sess.eegfile.duration
    windows = np.linspace(0,rec_duration,6)
    n_windows = len(windows)-1
    epochs = np.vstack([windows[:-1],windows[1:]]).T

    mean_frate = neurons.firing_rate[:,np.newaxis]

    frate = []
    for i,e in enumerate(epochs):
        if sess.artifact is not None: 
            artifact_duration = sess.artifact.time_slice(*e,strict=False).durations.sum()
        else:
            artifact_duration = 0
        n_spikes = neurons.time_slice(*e).n_spikes
        e_duration = e[1]-e[0]
        frate.append(n_spikes / (e_duration - artifact_duration))

    frate = np.array(frate).T 
    frate_perc = frate/mean_frate

    stable_bool = np.count_nonzero((frate_perc>(1/4)),axis=1)==n_windows
    stable_neurons = neurons[stable_bool]
    # sess.neurons_stable.save_with_date(sess.filePrefix.with_suffix('.neurons.stable'))
    stable_neurons.save(sess.filePrefix.with_suffix('.neurons.stable'))
    frate_all.append(frate_perc)

# frate_all = np.vstack(frate_all)

/data/Clustering/sessions/rolipram/BGN_2019-10-19_SDROL/BGN_2019-10-19_SDROL.neurons.stable saved
/data/Clustering/sessions/rolipram/BGU_2021-08-09_SDROL/BGU_2021-08-09_SDROL.neurons.stable saved
